# Execute this notebook first, after execute the notebook track_improved

In [1]:
# file path
artist_csv = "artists.csv"
artist_improved_intermediate="tracks_improved_intermediate.csv" # intermediate csv needed to run the second notebook
artist_improved_intermediate_dict = "artists_improved_intermediate.csv" # intermediate csv needed to run the second notebook

In [2]:
#import
import pandas
import numpy as np
import scipy.sparse as sp
import re
import collections
import tqdm

In [3]:
# read data
df = pandas.read_csv(filepath_or_buffer=artist_csv,sep="\t",header=0,
                usecols=['arid','artist_name','artist_uri'],
                dtype={'arid':np.int32,'artist_name':str, 'artist_uri':str})
df = df [['arid','artist_name','artist_uri']]
df.head()

,arid,artist_name,artist_uri
0,0,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk
1,1,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4
2,2,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m
3,3,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7
4,4,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij


In [4]:
#data
originals = df['artist_name'].values
artists = df['artist_name'].str.lower().values
arids = df['arid'].values
uris = df['artist_uri'].values
print(arids.shape[0])

295860


In [5]:
# split in main and co-artists (1st level)
def reg(vect):
    exp = ''
    for s in vect:
        exp += s + '|'
    exp = exp [:-1]
    return exp


def split_main_co_artists(value, reg):
    values = re.split(reg,str(value))
    l = len(values)
    main = []
    co = []
    if l == 1:
        main.append(values[0])
    elif l == 2:
        main.append(values[0])
        co.append(values [1])
    else:
        main.append(values [0])
        for i in range(1,l):
            co.append(values[i])
    return main, co    

#regex
s = []
s.append('\s[\(\[]?featuring[\.\:\.\,]?\s')
s.append('\s[\(\[]?featurin[\.\:\.\,]?\s')
s.append('\s[\(\[]?featured[\.\:\.\,]?\s')
s.append('\s[\(\[]?starring[\.\:\.\,]?\s')
s.append('\s[\(\[]?feat[\.\:\.\,]?\s')
s.append('\s[\(\[]?ft[\.\:\.\,]?\s')
s.append('\s[\(\[]?aka[\.\:\.\,]?\s')
s.append('\s[\(\[]?-[\.\:\.\,]?\s')
s.append('\s[\(\[]?introducing[\.\:\.\,]?\s')
s.append('\s[\(\[]?presents[\.\:\.\,]?\s')
s.append('\s[\(\[]?present[\.\:\.\,]?\s')
s.append('\s[\(\[]?duet\swith[\.\:\.\,]?\s')
s.append('\s[\(\[]?with[\.\:\.\,]?\s')
s.append('\sw\/\s')
s.append('\sf\/\s')
s.append('\s?\/\s?')
s.append('\s?\,\s\&\s?')
s.append('\smeets?\s')
s.append('\sand\shis\s')
s.append('\sand\sher\s')
s.append('\sand\sthem\s')
s.append('\s\&\shis\s')
s.append('\s\&\sher\s')
s.append('\s\&\sthem\s')
s.append('\s\&amp\;?\s')
s.append('[(|)]')
s.append('[\[|\]]')
s.append('[\{|\}]')
#spanish cases
s.append('\scon\sla\s')
s.append('\sy\ssus?\s')
s.append('\sy\slos?\s')
s.append('\spresenta\:?\s')
s.append('\scon\s')
s.append('\shaz\s')
#other lang
s.append('\smit\s')
s.append('\savec\s')
s.append('perf\.\s')
s.append('\slyr\.\s')
s.append('\sdir\.\s')
#special cases
s.append('\sfrom\:\s')
s.append('\sed\.\s')
s.append('\s?members\sof\sthe\s')
s.append('\s?members?\sof\s')
s.append('\svol\.?\s')
s.append('\s_\s')
s.append('performed\sby\s')
s.append('\spresents')
s.append('\s\'presents\'')
s.append('\spresents...')
s.append('\spresents\:')
s.append('\sfeaturng\s')
s.append('\sfeat\,')
s.append('[\(\[]feat[\.\:\.\,]')
s.append('feat\.')

reg_main_co_artists = reg(s)

c=0
main_a = []
co_a = []
for a in artists:
    main, co = split_main_co_artists(a,reg_main_co_artists)
    main_a.append(main)
    co_a.append(co)
    if len(co) + len(main) > 1:
        c += 1

if(len(main_a) != len(co_a)):
    print("ERROR")
else:
    print("DONE, found %d instances"%(c))

DONE, found 8068 instances


In [6]:
#split artists 2nd level (split main artists and after split the co-artists )

def split_artists(value, reg):
    artists = re.split(reg,str(value))
    return artists    

#regex
s = []
s.append('\sand\s')
s.append('\svs\.?')
s.append('\s?\-?conducted\sby\s')
s.append('\s?directed\sby\s')
s.append('\s?arranged\sby\s')
s.append('\sx\s')
s.append('\s\&\sco\.')
s.append('\s\&\s')
s.append('\s?\;\s?')
s.append('\s?\,\s?')
s.append('\s?\+\s?')
#spanish 
s.append('\sy\s')

reg_split_artists = reg(s)

main_a2 = []
co_a2 = []

# main artists
c1 = 0
for l_a in main_a:
    new_l = []
    for a in l_a:
        mains = split_artists(a,reg_split_artists)
        new_l = new_l + mains
        if len(mains)>1:
            #print (mains)
            c1 = c1 + 1
    main_a2.append(new_l)

# co-artists
c2 = 0
for l_a in co_a:
    new_l = []
    for a in l_a:
        co = split_artists(a,reg_split_artists)
        new_l = new_l + co
        if len(co)>1:
            #print (co)
            c2 = c2 + 1
    co_a2.append(new_l)

if(len(main_a2) != len(co_a2)):
    print("ERROR")
else:
    print("DONE, found %d instances (%d main artists, %d co-artists)"%(c1+c2,c1,c2))

DONE, found 18064 instances (17298 main artists, 766 co-artists)


In [19]:
#explore dataset
c=0
ens = main_a2.copy()
ens.append(co_a2)
for l in ens:
    for a in l:
        if "search a word" in a:
        #if len(a) > 40:
            c=c+1
            print (a)
print (c)


0


In [8]:
# class artist (attributes and a couple of utility methods)
class Artist:
    def __init__(self, original_artist, arid, uri, 
                 main_artists = [], co_artists = [], 
                 main_artists_ids = [], co_artists_ids = [], 
                ):
        self.arid = arid
        self.original_artist = original_artist
        self.main = main_artists
        self.co = co_artists
        self.main_ids = main_artists_ids
        self.co_ids = co_artists_ids
        self.uri = uri
        self.clean_names()
        self.shif_co_if_main_empty()
    def clean_names(self):
        self.main = list(map(str.strip, self.main))
        self.co = list(map(str.strip, self.co))
        self.main = list(filter(lambda s: s!='', self.main))
        self.co = list(filter(lambda s: s!='', self.co))
    def shif_co_if_main_empty(self):
        #shift first co in main if main is empty (happens when a name of the artist start with parenthesis)
        if len(self.co) != 0 and len(self.main) ==0:
            self.main.append(self.co[0])
            self.co = self.co[1:]
        # artist with no name, actually without filter single char happens just one time
        #if len(self.co) == 0 and len(self.main) == 0:
            #self.main.append('None')
    def reset_main_co_ids(self):
        self.main_ids = []
        self.co_ids = []
        

In [9]:
# create the artist objects
final_artists = []
for i in range (0,len(main_a2)):
    original = originals[i]
    main = main_a2[i]
    co = co_a2[i]
    uri = uris[i]
    arid = arids[i]
    final_artists.append(Artist(original,arid,uri,main,co))

In [10]:
# stat and search for attributes
def print_info_artist(a):
    print ("original: \t%s"%(a.original_artist))
    print ("main: \t\t%s"%(a.main))
    print ("co: \t\t%s"%(a.co))
    print ("main ids: \t%s"%(a.main_ids))
    print ("co ids: \t%s"%(a.co_ids))
    print ("id: \t\t%s"%(a.arid))
    print ("uri: \t\t%s"%(a.uri))
    return

count = 0
for a in final_artists:
    if a.arid == 135510:
        print_info_artist(a)
        count += 1
print(count)

original: 	Doc Holiday
main: 		['doc holiday']
co: 		[]
main ids: 	[]
co ids: 	[]
id: 		135510
uri: 		spotify:artist:7AzFGMYJyPu55Ea96Q4GY1
1


In [11]:
# build new ids for the artists

def get_new_id(name):
    global count
    if name not in new_dict:
        new_dict[name] = count
        count += 1
    return new_dict[name]

new_dict = {}
count = 0

for a in final_artists:
    a.reset_main_co_ids()
    for name in a.main:
        a.main_ids.append(get_new_id(name))
    for name in a.co:
        a.co_ids.append(get_new_id(name))


print ('new dictionary: %d artist'%(count))

new dictionary: 284238 artist


In [12]:
#TODO: clean artist which name is a stop word
# like: orquesta, orchestra, friends, karaoke, co., chorus, etc... (look stat analysis at the end for more details)

In [13]:
# write new data in a new csv 

artist_fields = ['arid','artist_uri','main_ids','co_ids','artist_name']#,'main_names','co_names']

full = []
for a in final_artists:
    row = []
    row.append(a.arid)
    row.append(a.uri)
    row.append(a.main_ids)
    row.append(a.co_ids)
    row.append(a.original_artist)
    #row.append(a.main)
    #row.append(a.co)
    full.append(row)

import csv
with open(artist_improved_intermediate, "w") as f:
    writer = csv.writer(f,delimiter = "\t",)
    writer.writerow(artist_fields)
    writer.writerows(full)
print (artist_improved_intermediate +" done")

artists_improved_intermediate.csv done


In [14]:
# build new dictionary usefull for the future work on extraction of artist in the tracks name
artist_fields = ['new_arid','new_artist_name']

inv_map = {v: k for k, v in new_dict.items()}

if len(inv_map)!=len(new_dict):
    print('ERROR conversion dictionary')

In [15]:
## write dict in csv    
import csv
full = []
for i in range(0,len(inv_map)):
    row = []
    row.append(i)
    row.append(inv_map[i])
    full.append(row)

with open(artist_improved_intermediate_dict, "w") as f:
    writer = csv.writer(f,delimiter = "\t",)
    writer.writerow(artist_fields)
    writer.writerows(full)
print ( artist_improved_intermediate_dict +" done")

artists_improved_intermediate_dict.csv done


In [16]:
#stat analysis

In [17]:
stats = {}
for a in inv_map:
    row = []
    row.append(0)
    row.append(inv_map[a])
    stats[a] = row

for a in final_artists:
    for i in a.main_ids:
        stats[i][0] +=1
    for i in a.co_ids:
        stats[i][0] +=1

In [18]:
for e in stats:
    a=stats[e]
    if a[0]>20:# and a[0]<25:
        print('%d\t\t%s'%(a[0],a[1]))

22		tyler ward
23		bing crosby
22		frank sinatra
79		chorus
82		jr.
37		alex
72		various artists
23		kim
28		jack
44		canton jones
79		company
28		family
160		friends
32		love
53		me
22		you
23		cast
577		orchestra
24		james
28		mike
21		grupo
28		combo
34		i
30		a
22		s
31		johnny
27		singers
31		lex de azevedo
21		performer
33		ensemble
23		chris
26		rdb
37		the
55		conjunto
127		orquesta
28		us
22		lata mangeshkar
21		adam
30		david
25		ghost
37		karaoke
54		band
25		relaxation
21		1
21		meditation
28		piano
22		bounty killer
22		woodie
22		beat rabbi
22		deepspace5


In [ ]:
#TODO now or in preprocessing, remove stopwords